In [ ]:
from Functionbase import *
from Backtest import *

In [ ]:
import os 
from dotenv import load_dotenv
import shioaji as sj
print(sj.__version__)

In [ ]:

load_dotenv()

api = sj.Shioaji(simulation=True)
api.login(
    os.getenv('API_KEY'),
    os.getenv('SECRET_KEY'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

connection = sqlite3.connect('data.db')

In [ ]:
api.activate_ca(
    ca_path = '/Users/ray881116/Documents/Ray_M/ekey/551/F229985348/S/Sinopac.pfx',
    ca_passwd = os.getenv('YOUR_CA_PASSWORD'),
    person_id = os.getenv('YOUR_PERSON_ID'),
)

In [ ]:
open_orders = {}
order_qty = {} #下單張數 
positions = {}

In [ ]:
def place_cb(stat, msg):

    print('stat', stat)
    print('msg', msg)
    #'TFTDEAL' 下單成功
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


Buy order

In [ ]:
code = '2834'
current_price = 15.30
qty = 1

contract = api.Contracts.Stocks[code]

order = api.Order(price=current_price,
                  quantity=qty,
                  action="Buy",
                  price_type="LMT",
                  order_type="ROD",
                  account=api.stock_account)

trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = qty

print(open_orders)
print(order_qty)
print(positions)

In [ ]:
print(open_orders)
print(order_qty)
print(positions)

Sell order

In [ ]:
position = positions.get(code, 0)

current_price = 16

contract = api.Contracts.Stocks[code]

order = api.Order(price=current_price,
                  quantity=position,
                  action="Sell",
                  price_type="LMT",
                  order_type="ROD",
                  account=api.stock_account)

trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = position

print(open_orders)
print(order_qty)
print(positions)

In [ ]:
print(open_orders)
print(order_qty)
print(positions)

Cancel order

In [ ]:
code = '2834'
current_price = 15.8
qty = 1

contract = api.Contracts.Stocks[code]

order = api.Order(price=current_price,
                  quantity=qty,
                  action="Buy",
                  price_type="LMT",
                  order_type="ROD",
                  account=api.stock_account)

trade = api.place_order(contract, order)

In [ ]:
trade.status.status
api.update_status(api.stock_account)
trade.status.status

In [ ]:
api.cancel_order(trade)
trade.status.status

In [ ]:
api.update_status(api.stock_account)
trade.status.status

整合

In [ ]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions ={}

@api.quote.on_quote
def qoute_callback(topic:str, qoute:dict):
    
    code = topic.split('/')[-1]

    ts = pd.to_datetime(qoute.get('Date')+ '' +qoute.get('Time'))

    if ts.hour < 9:
        return
    
    close = qoute.get('Close')[0]
    volume = qoute.get('Volume')[0]
    vol_sum = qoute.get('Volsum')[0]

    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

def place_cb(stat, msg):

    if stat.value == 'TETDEAL':

        code = msg['code']
    qty = int(msg['quantity'])

    order_qty[code] -= qty

    if order_qty[code] == 0:
        open_orders[code] = None

    position = positions.get(code,0)

    if msg['action'] == 'Sell':
        qyt = qty * -1

    position += qty
    positions[code] = position

api.set_order_callback(place_cb)


In [ ]:
import time

def tbot(code):

    print(" ----- 啟動 Tbot  #{} -----".format(code))

    tw_calendar = get_calendar('XTAI')

    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    ticks[code] = pd.concat([get_ticks(code, prev_trading_date, connection, api)[0], get_ticks(code, date, connection, api)[0]])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()

    api.quote.subscribe(api.Contracts.Stocks[code], quote_type = 'tick')

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0:

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
            kbars[code] = kbars[code][date:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')
            
            try:
                
                current_price = kbars[code].loc[str_current_time]['close']
            
            except:
                continue

            open_order = open_orders.get(code, None)

            if open_order:

                api.update_status(api.stock_account)

                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:

                    api.cancel_order(open_order)

                    open_orders[code] = None
                    order_qty[code] = 0

                    if open_order.status.status != 'Filling':
                        break

            position = positions.get(code, 0)

            if(
                current_time >= current_time.replace(hour=9, minute=15, second=0) and
                current_time <= current_time.replace(hour=9, minute=30, second=0) and
                position == 0
            ):
                
                high_15m = kbars[code][:current_time.replace(hour=9, minute=14, second=0)]['high'].max()
                low_15m =  kbars[code][:current_time.replace(hour=9, minute=14, second=0)]['high'].min()

                if (
                    current_price > high_15m and 
                    volume_today[code] >= 2000
                ):
                    
                    qty = 1

                    contract = api.Contracts.Stocks[code]
                    order = api.Order(
                        price = current_price,
                        quantity = qty,
                        action = 'Buy',
                        price_type = 'LMT',
                        order_type= 'ROD',
                        account = api.stock_account
                    )

                    trade = api.place_order( contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty
                    target_price = current_price * 1.03
                    stop_price = low_15m

                    print('[{}] buy {} at {}'.format(str_current_time, code, current_price))

                #達停利停損條件
                elif(
                    current_time >= current_time.replace(hour= 9, minute= 15, second=0) and
                    current_time <= current_time.replace(hour=13, minute=15, second=0) and 
                    position !=0
                ):
                    if (
                        current_price >= target_price or
                        current_price <= stop_price
                    ):
                        
                        contract = api.Contracts.Stocks[code]
                        order = api.Order(
                            price= current_price,
                            quantity= position,
                            action= 'Sell',
                            price_type= 'LMT',
                            order_type= 'ROD',
                            account = api.stock_account
                        )

                        trade = api.place_order(contract, order)

                        open_orders[code] = trade
                        order_qty[code] = position

                        print('[{}] sell {} at {}'.format(str_current_time, code, current_price))
                        break
                    #未達條件，當日出場
                    elif(
                        current_time >= current_time.replace( hour=13, minute= 15, second=0)
                    ):
                        if position != 0:

                            contract = api.Contracts.Stocks[code]
                            order = api.Order(
                                price= current_price,
                                quantity= position,
                                action= 'Sell',
                                price_type= 'LMT',
                                order_type= 'ROD',
                                account = api.stock_account
                            )

                            trade = api.place_order( contract, order)

                            open_orders[code] = trade
                            order_qty[code] = position

                            print('[{}] sell {} at {}'.format(str_current_time, code, current_price))
                        
                        break
api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type = 'tick')

print('----- 關閉 Tbot  #{} -----'.format(code))

In [ ]:
tbot('3041')